### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
import os
# Load environment variables from .env file
load_dotenv()  
# Retrieve the API key
NASA_API_KEY = os.getenv("NASA_API_KEY")
  




### CME Data

In [3]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"


# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"




In [4]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(url)





In [5]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()





In [6]:
# Preview the first result in JSON format

# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[10], indent=4, sort_keys=True))




{
    "activeRegionNum": null,
    "activityID": "2013-05-13T02:54:00-CME-001",
    "catalog": "M2M_CATALOG",
    "cmeAnalyses": [
        {
            "enlilList": [
                {
                    "au": 2.0,
                    "cmeIDs": [
                        "2013-05-13T02:54:00-CME-001"
                    ],
                    "estimatedDuration": null,
                    "estimatedShockArrivalTime": null,
                    "impactList": null,
                    "isEarthGB": false,
                    "kp_135": null,
                    "kp_18": null,
                    "kp_180": null,
                    "kp_90": null,
                    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/WSA-ENLIL/2416/-1",
                    "modelCompletionTime": "2013-05-13T01:36Z",
                    "rmin_re": null
                },
                {
                    "au": 2.0,
                    "cmeIDs": [
                        "2013-05-13T02:54:00-CME-001",

In [7]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)


# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[["activityID", "startTime", "linkedEvents"]]
print(cme_df.head())


                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-02T05:24:00-CME-001  2013-05-02T05:24Z   
2  2013-05-02T14:36:00-CME-001  2013-05-02T14:36Z   
3  2013-05-03T18:00:00-CME-001  2013-05-03T18:00Z   
4  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   

                                      linkedEvents  
0  [{'activityID': '2013-05-04T04:52:00-IPS-001'}]  
1                                             None  
2                                             None  
3                                             None  
4  [{'activityID': '2013-05-07T04:37:00-IPS-001'}]  


In [8]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# We will need to extract the GST from the linkedEvents column
# Let's preview the first row in the linkedEvents column


# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=["linkedEvents"])
print(cme_df.head())


                     activityID          startTime  \
0   2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
4   2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
7   2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
10  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
13  2013-05-13T16:18:00-CME-001  2013-05-13T16:18Z   

                                         linkedEvents  
0     [{'activityID': '2013-05-04T04:52:00-IPS-001'}]  
4     [{'activityID': '2013-05-07T04:37:00-IPS-001'}]  
7     [{'activityID': '2013-05-12T23:30:00-IPS-001'}]  
10  [{'activityID': '2013-05-13T01:53:00-FLR-001'}...  
13  [{'activityID': '2013-05-13T15:40:00-FLR-001'}...  


In [9]:
# Notice that the linkedEvents sometimes contains multiple events per row
# We will need to expand these rows into multiple rows
# Let's preview the first row in the linkedEvents column



# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' column for each row
# and adds the elements individually to a list of dictionaries where each row is one element 
# with the corresponding 'activityID' and 'startTime' value



# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for index in cme_df.index:
    activityID = cme_df.loc[index,"activityID"]
    startTime = cme_df.loc[index,"startTime"]
    linkedEvents = cme_df.loc[index,"linkedEvents"]

    # Iterate over each dictionary in the list
    for item in linkedEvents:
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({"activityID": activityID, "startTime": startTime, "linkedEvent": item})
     
      
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)
print(expanded_df.head())


                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                     linkedEvent  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  


In [10]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(dict):
        try:
            activityID = dict.get("activityID", None)
            return activityID

        # Log the error or print it for debugging
        except KeyError:
            print(f"Error: {dict}")
            return None
        
# Test the function with one row from the linkedEvents column
print(extract_activityID_from_dict(expanded_df.loc[0,"linkedEvent"]))





2013-05-04T04:52:00-IPS-001


In [11]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
expanded_df.loc[:,"GST_ActivityID"] = expanded_df["linkedEvent"].apply(lambda x: extract_activityID_from_dict(x))
# and create a new column called 'GST_ActivityID' using loc indexer:
print(expanded_df.head())



                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                     linkedEvent               GST_ActivityID  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  2013-05-04T04:52:00-IPS-001  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  2013-05-07T04:37:00-IPS-001  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  2013-05-12T23:30:00-IPS-001  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  2013-05-13T01:53:00-FLR-001  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  2013-05-13T04:12:00-SEP-001  


In [12]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_df = expanded_df.dropna(subset=["GST_ActivityID"])
print(expanded_df.head())



                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                     linkedEvent               GST_ActivityID  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  2013-05-04T04:52:00-IPS-001  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  2013-05-07T04:37:00-IPS-001  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  2013-05-12T23:30:00-IPS-001  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  2013-05-13T01:53:00-FLR-001  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  2013-05-13T04:12:00-SEP-001  


In [13]:
# print out the datatype of each column in this DataFrame:
print(expanded_df.dtypes)



activityID        object
startTime         object
linkedEvent       object
GST_ActivityID    object
dtype: object


In [14]:
# Convert the 'GST_ActivityID' column to string format 
expanded_df["GST_ActivityID"] = expanded_df["GST_ActivityID"].astype("string")


# Convert startTime to datetime format 
expanded_df["startTime"] = pd.to_datetime(expanded_df["startTime"])


# Rename startTime to startTime_CME and activityID to cmeID
expanded_df = expanded_df.rename(columns={"startTime": "startTime_CME", "activityID": "cmeID"})


# Drop linkedEvents
expanded_df = expanded_df.drop(columns=["linkedEvent"])


# Verify that all steps were executed correctly
print(expanded_df.dtypes)



cmeID                          object
startTime_CME     datetime64[ns, UTC]
GST_ActivityID         string[python]
dtype: object


In [15]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
expanded_df = expanded_df[expanded_df["GST_ActivityID"].str.contains("GST")]
print(expanded_df.head())










                           cmeID             startTime_CME  \
21   2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00   
48   2013-09-29T22:40:00-CME-001 2013-09-29 22:40:00+00:00   
90   2013-12-04T23:12:00-CME-001 2013-12-04 23:12:00+00:00   
148  2014-02-16T14:15:00-CME-001 2014-02-16 14:15:00+00:00   
151  2014-02-18T01:25:00-CME-001 2014-02-18 01:25:00+00:00   

                  GST_ActivityID  
21   2013-06-07T03:00:00-GST-001  
48   2013-10-02T03:00:00-GST-001  
90   2013-12-08T00:00:00-GST-001  
148  2014-02-19T03:00:00-GST-001  
151  2014-02-20T03:00:00-GST-001  


### GST Data

In [16]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"


# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"


# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"


# Build URL for GST
url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"



In [17]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(url)




In [18]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()


# Preview the first result in JSON format


# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[10], indent=4 ))




{
    "gstID": "2014-09-12T15:00:00-GST-001",
    "startTime": "2014-09-12T15:00Z",
    "allKpIndex": [
        {
            "observedTime": "2014-09-12T18:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        },
        {
            "observedTime": "2014-09-13T00:00Z",
            "kpIndex": 7.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/6639/-1",
    "linkedEvents": [
        {
            "activityID": "2014-09-10T18:18:00-CME-001"
        },
        {
            "activityID": "2014-09-12T15:26:00-IPS-001"
        }
    ],
    "submissionTime": "2014-09-12T18:11Z",
    "versionId": 1
}


In [19]:
gst_df = pd.DataFrame(gst_json)
print(gst_df.head())

                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-06-29T03:00:00-GST-001  2013-06-29T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   

                                          allKpIndex  \
0  [{'observedTime': '2013-06-01T01:00Z', 'kpInde...   
1  [{'observedTime': '2013-06-07T03:00Z', 'kpInde...   
2  [{'observedTime': '2013-06-29T03:00Z', 'kpInde...   
3  [{'observedTime': '2013-10-02T06:00Z', 'kpInde...   
4  [{'observedTime': '2013-12-08T03:00Z', 'kpInde...   

                                                link  \
0  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   
1  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   
2  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   
3  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   
4  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   

       

In [20]:
gst_df = gst_df[["gstID", "startTime", "linkedEvents"]]
print(gst_df.head())






                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-06-29T03:00:00-GST-001  2013-06-29T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   

                                        linkedEvents  
0    [{'activityID': '2013-05-31T15:45:00-HSS-001'}]  
1    [{'activityID': '2013-06-02T20:24:00-CME-001'}]  
2                                               None  
3  [{'activityID': '2013-09-29T22:40:00-CME-001'}...  
4  [{'activityID': '2013-12-04T23:12:00-CME-001'}...  


In [21]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# We will need to extract the CME from the linkedEvents column
# Let's preview the first row in the linkedEvents column


# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=["linkedEvents"])
print(gst_df.head())


                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   
5  2014-02-19T03:00:00-GST-001  2014-02-19T03:00Z   

                                        linkedEvents  
0    [{'activityID': '2013-05-31T15:45:00-HSS-001'}]  
1    [{'activityID': '2013-06-02T20:24:00-CME-001'}]  
3  [{'activityID': '2013-09-29T22:40:00-CME-001'}...  
4  [{'activityID': '2013-12-04T23:12:00-CME-001'}...  
5  [{'activityID': '2014-02-16T14:15:00-CME-001'}...  


In [22]:
# Notice that the linkedEvents sometimes contains multiple events per row
# We will need to expand these rows into multiple rows
# Let's preview the first row in the linkedEvents column
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode("linkedEvents",ignore_index=True).dropna()
print(gst_df)


                           gstID          startTime  \
0    2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1    2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
..                           ...                ...   
200  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
201  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
202  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
203  2024-04-19T18:00:00-GST-001  2024-04-19T18:00Z   
204  2024-04-19T18:00:00-GST-001  2024-04-19T18:00Z   

                                      linkedEvents  
0    {'activityID': '2013-05-31T15:45:00-HSS-001'}  
1    {'activityID': '2013-06-02T20:24:00-CME-001'}  
2    {'activityID': '2013-09-29T22:40:00-CME-001'}  
3    {'activityID': '2013-10-02T01:54:00-IPS-001'}  
4    {'activityID': '2013-10-02T02:47:00-MPC-001'}  
..                   

In [23]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
gst_df.loc[:,"CME_ActivityID"] = gst_df["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))
# and create a new column called 'CME_ActivityID' using loc indexer:



# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df.dropna(subset=["CME_ActivityID"])
print(gst_df)



                           gstID          startTime  \
0    2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1    2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
..                           ...                ...   
200  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
201  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
202  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
203  2024-04-19T18:00:00-GST-001  2024-04-19T18:00Z   
204  2024-04-19T18:00:00-GST-001  2024-04-19T18:00Z   

                                      linkedEvents  \
0    {'activityID': '2013-05-31T15:45:00-HSS-001'}   
1    {'activityID': '2013-06-02T20:24:00-CME-001'}   
2    {'activityID': '2013-09-29T22:40:00-CME-001'}   
3    {'activityID': '2013-10-02T01:54:00-IPS-001'}   
4    {'activityID': '2013-10-02T02:47:00-MPC-001'}   
..             

In [24]:
# Convert the 'CME_ActivityID' column to string format 
# Check if 'CME_ActivityID' column exists in the DataFrame before converting it to string format

gst_df["CME_ActivityID"] = gst_df["CME_ActivityID"].astype("string")

# Verify that all steps were executed correctly

gst_df["startTime"] = pd.to_datetime(gst_df["startTime"])


# Rename startTime to startTime_GST
gst_df = gst_df.rename(columns={"startTime": "startTime_GST"})

# Drop linkedEvents
gst_df = gst_df.drop(columns=["linkedEvents"])

gst_df["gstID"] = gst_df["gstID"].astype("string")


# Verify that all steps were executed correctly
print(gst_df.dtypes)








gstID                  string[python]
startTime_GST     datetime64[ns, UTC]
CME_ActivityID         string[python]
dtype: object


In [31]:
# Check if 'CME_ActivityID' column exists in the DataFrame before filtering

gst_df = gst_df[gst_df["CME_ActivityID"].str.contains("CME")]
print(gst_df)
  


                           gstID             startTime_GST  \
1    2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
2    2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
5    2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
7    2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
9    2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   
..                           ...                       ...   
192  2023-12-18T06:00:00-GST-001 2023-12-18 06:00:00+00:00   
195  2024-03-03T18:00:00-GST-001 2024-03-03 18:00:00+00:00   
199  2024-03-24T12:00:00-GST-001 2024-03-24 12:00:00+00:00   
200  2024-03-24T12:00:00-GST-001 2024-03-24 12:00:00+00:00   
203  2024-04-19T18:00:00-GST-001 2024-04-19 18:00:00+00:00   

                  CME_ActivityID  
1    2013-06-02T20:24:00-CME-001  
2    2013-09-29T22:40:00-CME-001  
5    2013-12-04T23:12:00-CME-001  
7    2014-02-16T14:15:00-CME-001  
9    2014-02-18T01:25:00-CME-001  
..                           ...  
192  2023-

### Merge both datatsets

In [32]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.

merged_df = pd.merge(gst_df, expanded_df, left_on=['gstID', 'CME_ActivityID'], right_on=['GST_ActivityID', 'cmeID'])


print(merged_df)



                          gstID             startTime_GST  \
0   2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1   2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2   2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3   2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4   2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   
..                          ...                       ...   
56  2023-12-18T06:00:00-GST-001 2023-12-18 06:00:00+00:00   
57  2024-03-03T18:00:00-GST-001 2024-03-03 18:00:00+00:00   
58  2024-03-24T12:00:00-GST-001 2024-03-24 12:00:00+00:00   
59  2024-03-24T12:00:00-GST-001 2024-03-24 12:00:00+00:00   
60  2024-04-19T18:00:00-GST-001 2024-04-19 18:00:00+00:00   

                 CME_ActivityID                        cmeID  \
0   2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1   2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2   2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3   2014-02

In [34]:
# Verify that the new DataFrame has the same number of rows as cme and gst DataFrames
merged_df.info()






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     string             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  61 non-null     object             
 3   cmeID           61 non-null     object             
 4   startTime_CME   61 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  61 non-null     string             
dtypes: datetime64[ns, UTC](2), object(2), string(2)
memory usage: 3.0+ KB


### Computing the time it takes for a CME to cause a GST

In [35]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df["timeDiff"] = merged_df["startTime_GST"] - merged_df["startTime_CME"]
print(merged_df)


                          gstID             startTime_GST  \
0   2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1   2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2   2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3   2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4   2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   
..                          ...                       ...   
56  2023-12-18T06:00:00-GST-001 2023-12-18 06:00:00+00:00   
57  2024-03-03T18:00:00-GST-001 2024-03-03 18:00:00+00:00   
58  2024-03-24T12:00:00-GST-001 2024-03-24 12:00:00+00:00   
59  2024-03-24T12:00:00-GST-001 2024-03-24 12:00:00+00:00   
60  2024-04-19T18:00:00-GST-001 2024-04-19 18:00:00+00:00   

                 CME_ActivityID                        cmeID  \
0   2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1   2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2   2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3   2014-02

In [36]:
# Use describe() to compute the mean and median time 
print(merged_df["timeDiff"].describe())




count                           61
mean     2 days 23:29:26.557377049
std      0 days 23:53:09.336914240
min                1 days 08:36:00
25%                2 days 04:00:00
50%                2 days 19:51:00
75%                3 days 13:35:00
max                6 days 03:00:00
Name: timeDiff, dtype: object


### Exporting data in csv format

In [37]:
# Export data to CSV without the index
merged_df.to_csv("merged_df.csv", index=False)


